In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Load dataset with appropriate encoding to avoid decoding errors
df = pd.read_csv('figma_dataset2.csv', encoding='latin1')

In [3]:
# Display first few rows to understand the structure
print(df.head())

      tag   type       x       y  width   height characters  depth  \
0    BODY  FRAME       0       0  800.0  10577.0        NaN      0   
1     DIV  GROUP       0       0  800.0      0.0        NaN      1   
2     DIV  GROUP -100000 -100000  800.0    600.0        NaN      2   
3  IFRAME  GROUP -100000 -100000  800.0    600.0        NaN      3   
4     DIV  GROUP       0       0  800.0  10577.0        NaN      1   

   num_children parent_tag  ...  x_normalized  y_normalized  x_center  \
0             3        NaN  ...        100000        100000     400.0   
1             1       BODY  ...        100000        100000     400.0   
2             1        DIV  ...             0             0  -99600.0   
3             0        DIV  ...             0             0  -99600.0   
4             1       BODY  ...        100000        100000     400.0   

  y_center x_quarter y_quarter  aspect_ratio       area normalized_width  \
0   5288.5       0.5  0.500000      0.075636  8461600.0         

In [4]:
# Define column categories based on the dataset attributes
categorical_cols = ['type', 'characters', 'parent_tag', 'text_type', 'visibility', 'border_type', 'border_pattern', 'shadow_type']
numerical_cols = ['x', 'y', 'width', 'height', 'depth', 'num_children', 'sibling_count', 'is_leaf', 'font_size', 'font_weight',
                   'color', 'background_color', 'border_radius', 'border_color', 'border_opacity', 'border_weight', 'shadow_color',
                   'shadow_offset', 'shadow_radius', 'text_length', 'word_count', 'contains_number', 'contains_special_chars',
                   'x_normalized', 'y_normalized', 'x_center', 'y_center', 'x_quarter', 'y_quarter', 'aspect_ratio', 'area',
                   'normalized_width', 'normalized_height']

In [5]:
# Drop non-numeric values from numerical columns to prevent conversion errors
df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric, errors='coerce')

In [6]:
# Fill missing values with appropriate replacements
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

In [7]:
# Ensure all categorical columns are treated as strings before encoding
df[categorical_cols] = df[categorical_cols].astype(str)

In [8]:
# Encode categorical features using Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
# Encode target variable
tag_encoder = LabelEncoder()
df['tag'] = tag_encoder.fit_transform(df['tag'])

In [10]:
# Split dataset into features and target
X = df.drop(columns=['tag'])  # Features
y = df['tag']  # Target variable

In [ ]:
# Replace infinite values with NaN
X[numerical_cols] = X[numerical_cols].replace([np.inf, -np.inf], np.nan)

# Normalize numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

C:\Users\AOZ\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\AOZ\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\AOZ\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check if a pre-trained model exists
model_filename = "models/html_tag_model.pkl"
scaler_filename = "models/scaler.pkl"
tag_encoder_filename = "models/tag_encoder.pkl"
label_encoders_filename = "models/label_encoders.pkl"

try:
    model = joblib.load(model_filename)
    scaler = joblib.load(scaler_filename)
    tag_encoder = joblib.load(tag_encoder_filename)
    label_encoders = joblib.load(label_encoders_filename)
    print("Loaded pre-trained model.")
except FileNotFoundError:
    print("No pre-trained model found, training a new one.")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Save the trained model, scaler, and encoder
    joblib.dump(model, model_filename)
    joblib.dump(scaler, scaler_filename)
    joblib.dump(tag_encoder, tag_encoder_filename)
    joblib.dump(label_encoders, label_encoders_filename)
    print("Model saved for future use.")

In [ ]:
# Predict and evaluate model performance
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred, labels=np.unique(y_test), target_names=tag_encoder.inverse_transform(np.unique(y_test))))